![](https://www.ieseg.fr/wp-content/uploads/IESEG-Logo-2012-rgb.jpg)


In [2]:
import pandas as pd
import numpy as np
import plotly.express as px

In [3]:
#conda install -c conda-forge scikit-surprise

# Collaborative Filtering

### Exercise 1

- Read the Amazon Reviews dataset (<font color="blue">Amazon_Reviews_2019.csv</font>)
- Filter the data to only contain items with at least 10 reviews and user with at least 3 reviews.
- Pivot the dataframe so that the reviewer names (or ids) are in the rows, the items are in the columns and the values contain the user-item ratings
- compute the sparsity (percentage of NA values) of the pivoted dataset

In [4]:
df = pd.read_csv (r'C:\Users\fdelgado\OneDrive - IESEG\Documents\01. IESEG\16. Recommendation Systems\Session 1\Amazon_Reviews_2019.csv')

In [5]:
df.head()

,reviewerID,dateAdded,dateUpdated,name,asin,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,100000,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,B00QWO9P0O,Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,100001,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,B00QWO9P0O,Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,100002,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,B00QWO9P0O,Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,100003,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,B00QWO9P0O,Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,100004,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,B00QWO9P0O,Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [6]:
#reviews by product 
avg_products = df.groupby('asin').agg("count")["dateAdded"]
avg_products = pd.DataFrame(avg_products)    
avg_products = avg_products.rename(columns={'dateAdded':'review_count'})

#reviews by user 
avg_users = df.groupby('reviewerID').agg("count")["dateAdded"]
avg_users = pd.DataFrame(avg_users)    
avg_users = avg_users.rename(columns={'dateAdded':'review_count'})

In [7]:
#subset products with more than 9 reviews 
products = avg_products[avg_products['review_count'] > 9]
products.head()

#subset users with more than 9 reviews 
users = avg_users[avg_users['review_count'] > 2]
users.head()

,review_count
reviewerID,
100001,7
100004,3
100006,889
100015,3
100017,12


In [8]:
#put index into column
users['id'] = users.index
products['id'] = products.index

<ipython-input-8-05778fac969d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users['id'] = users.index
<ipython-input-8-05778fac969d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products['id'] = products.index


In [9]:
#Filter out users
df_filter = df.loc[df['reviewerID'].isin(users['id'])]
df_filter = df_filter.loc[df_filter['asin'].isin(products['id'])]

In [10]:
#We have 34 unique products
df_filter.asin.nunique()

#We have 2171 unique users
df_filter.reviewerID.nunique()

2171

In [11]:
#pivot table
new_table = pd.pivot_table(df_filter, values='reviews.rating', index="reviewerID",
                    columns=['asin'])
new_table.head()

asin,B00DUGZFWY,B00EEBS9O0,B00IOY8XWQ,B00IOYAM4I,B00QFQRELG,B00QWO9P0O,B00REQKWGA,B00VINDBJK,B00WRDS8H0,B00ZS0G0PG,...,B01AHBBG04,B01AHBDCKQ,B01BH83OOM,B01IO618J8,B01J94SBEY,B01J94SCAM,B01J94SWWU,B01J94T1Z2,B01J94YIT6,B06XD5YCKX
reviewerID,,,,,,,,,,,,,,,,,,,,,
100001,NaN,NaN,NaN,NaN,NaN,4.714286,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100004,NaN,NaN,NaN,NaN,NaN,5.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100006,NaN,NaN,NaN,NaN,NaN,4.350956,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100015,NaN,NaN,NaN,NaN,NaN,2.333333,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100017,NaN,NaN,NaN,NaN,NaN,4.916667,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
percent_missing = new_table.isnull().sum() * 100 / len(new_table)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})

In [13]:
#Sparsity
missing_value_df.percent_missing.mean()

90.51805890481481

### Exercise 2

 - Split the dataset (not pivoted) in train and test data based on the "reviews.date" column
 - Cutoff date: 01st May 2017
 - Executing the cell below should result in:

 **Train**
- Nr items: 27
- Nr users: 2065

**Test**
- Nr items: 28
- Nr users: 470

**New**
- Nr new items: 7
- Nr new users: 105


In [14]:
#Clean date column 
df_filter['reviews.date'] = pd.to_datetime(df_filter['reviews.date'])
df_filter['reviews.date'] = df_filter['reviews.date'].astype(str)
new = df_filter["reviews.date"].str.split(" ", n = 1, expand = True)
df_filter['reviews.date'] = new[0]
df_filter['reviews.date'] = pd.to_datetime(df_filter['reviews.date'])

In [15]:
#Filter anything 
train = df_filter[df_filter['reviews.date']<pd.to_datetime("2017-05-01")]
test = df_filter[df_filter['reviews.date']>=pd.to_datetime("2017-05-01")]

In [16]:
#Validate
train['reviews.date'].max()

Timestamp('2017-04-29 00:00:00')

In [17]:
print('------ Train ------')
print('Nr items:', len(train["asin"].unique()))
print('Nr users:', len(train["reviews.username"].unique()))

print('\n------ Test ------')
print('Nr items:', len(test["asin"].unique()))
print('Nr users:', len(test["reviews.username"].unique()))

print('\nNr new items:', len(set(test["asin"])-set(train["asin"])))
print('Nr new users:', len(set(test["reviews.username"])-set(train["reviews.username"])))

------ Train ------
Nr items: 27
Nr users: 2066

------ Test ------
Nr items: 28
Nr users: 470

Nr new items: 7
Nr new users: 105


### Exercise 3

- Fit KNNBasic using <font color="blue">cosine similarity, k=20, min_k=5, user_based=True</font> on the train set 
- Compute the predictions for the test set
- Evaluate the predictions based on RMSE (surprise.accuracy.rmse)

In [18]:
from surprise import Dataset
from surprise import Reader
from surprise import KNNBasic , KNNWithMeans , KNNWithZScore , KNNBaseline

In [19]:
train.head()

,reviewerID,dateAdded,dateUpdated,name,asin,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
1,100001,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,B00QWO9P0O,Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
4,100004,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,B00QWO9P0O,Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."
6,100006,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,B00QWO9P0O,Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,ive not had any problame with these batteries ...,... batteries have ordered them in the past be...,ByAmazon Customer,"https://www.barcodable.com/upc/841710106442,ht..."
15,100006,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,B00QWO9P0O,Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Thankful that I was able to find on Amazon for...,... I was able to find on Amazon for a great p...,ByAmazon Customer,"https://www.barcodable.com/upc/841710106442,ht..."
16,100015,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,B00QWO9P0O,Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,1,https://www.amazon.com/product-reviews/B00QWO9...,I don't know if I would buy thus brand again s...,... know if I would buy thus brand again seems...,ByLaura,"https://www.barcodable.com/upc/841710106442,ht..."


In [20]:
#subset train and test with 3 columns
train = train[['reviewerID', 'asin', 'reviews.rating']]
test = test[['reviewerID', 'asin', 'reviews.rating']]

In [21]:
# set up reader with min rating and max rating as arguments
reader = Reader(rating_scale=(1 , 5))
# surprise training and test set
df_train = Dataset.load_from_df(train , reader).build_full_trainset()
df_test = list(test.itertuples(index=False , name=None))

In [27]:
#Init model
model = KNNBasic(k = 20, min_k = 5 , user_based = True)
# fit on training set
model.fit(df_train)

model_pred = model.test(df_test)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [28]:
from surprise import accuracy
accuracy.rmse(model_pred)

RMSE: 0.9485


0.9484988746228629

### Exercise 4

- Fit KNNBasic using <font color="blue">pearson correlation, k=10, min_k=2, user_based=False</font> on the train set 
- Compute the predictions for the test set
- Evaluate the predictions based on RMSE (surprise.accuracy.rmse)

In [29]:
#Init model
model = KNNBasic(k = 10, min_k = 2 , user_based = False)
# fit on training set
model.fit(df_train)

model_pred = model.test(df_test)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [30]:
from surprise import accuracy
accuracy.rmse(model_pred)

RMSE: 0.9565


0.9564987344446368

### Exercise 5

- Fit ALS on the train set (select your own parameter)
- Fit SVD on the train set (select your own parameter)
- Compute the predictions for the test set
- Evaluate the predictions based on RMSE (surprise.accuracy.rmse)

In [36]:
from surprise import BaselineOnly , SVD, CoClustering

In [46]:
options = {"method": "als", "n_epochs": 7}
als = BaselineOnly(bsl_options=options)

# fit on training set
als.fit(df_train)

# predict test set
als_pred = als.test(df_test)

# compute rmse
accuracy.rmse(als_pred)

Estimating biases using als...
RMSE: 0.8777


0.8777160213195775

In [48]:
svd = SVD(n_factors=5,random_state=42)

# fit on training set
svd.fit(df_train)

# predict test set
svd_pred = svd.test(df_test)

# compute rmse
accuracy.rmse(svd_pred)

RMSE: 0.8912


0.891161694529738

### Exercise 6

- Fit CoClustering using <font color="blue">user clusters=10, item clusters=10, epochs=50</font> on the train set 
- Compute the predictions for the test set
- Evaluate the predictions based on RMSE (surprise.accuracy.rmse)

In [50]:
clust = CoClustering(n_cltr_u= 10 , n_cltr_i = 10,n_epochs = 50 , random_state = 42)

# fit on training set
clust.fit(df_train)

# predict test set
clust_pred = clust.test(df_test)

# compute rmse
accuracy.rmse(clust_pred)

<ipython-input-50-fa797124a455>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  clust.fit(df_train)


RMSE: 0.9808


0.9807687627816852

### Exercise 7

- Display the Top 3 recommendations for the user "Robert" for all models (user-based, item-based, svd, clustering)
- you can use the function below

```python

def predict_user_topn(model, train, user, topk=15, item_col='item'):
    """
    Returns topk items for specified user.
    Return type: list

    Args[model, train, user, topk, item_col]
    model:      fitted model (surprise package)
    train:      train set used to fit model
    user:       user id
    topk:       topk items to return
    item_col:   column containing item ids 
    """

    # intermediate result dict 
    res = {item_col:[], 'pred':[], 'detail':[]}
    
    # iter through all items contained in train set
    for item in set(train[item_col]) :
        uid, iid, true_r, est, detail = model.predict(user,item)
        if detail['was_impossible']: continue
        # save to result dict
        res[item_col].append(item)
        res['pred'].append(est)
        res['detail'].append(detail)

    return list(pd.DataFrame(res).sort_values('pred', ascending=False)[:topk][item_col])

```

In [88]:
models = [svd, als, clust]

#Loop into dictionary
d={}
for model in models:
    d[model] = predict_user_topn(model, train, "Robert", topk=15, item_col="asin")
    
#Create dataframe
d = pd.DataFrame.from_dict(d)
d.rename(columns={ d.columns[0]: "SVD", d.columns[1]: "ALS", d.columns[2]: "Clustering"}, inplace = True)
d.index += 1 

In [89]:
d

,SVD,ALS,Clustering
1,B00QFQRELG,B00QFQRELG,B018Y224PY
2,B00IOY8XWQ,B00IOY8XWQ,B018Y226XO
3,B00IOYAM4I,B00IOYAM4I,B00VINDBJK
4,B018SZT3BK,B018SZT3BK,B00EEBS9O0
5,B018T075DC,B00REQKWGA,B00IOY8XWQ
6,B00REQKWGA,B018T075DC,B018Y23MNM
7,B00ZS0G0PG,B00ZS0G0PG,B018Y225IA
8,B0189XZRTI,B01AHBDCKQ,B01BH83OOM
9,B01AHBBG04,B0189XZRTI,B018Y22BI4
10,B00VINDBJK,B01AHB9C1E,B00ZV9PXP2
